## List of all RV planets in the exoplanets archive less than 70pc retreived from online interface

In [24]:
import warnings
warnings.filterwarnings('ignore')
from astroquery.ipac.nexsci.nasa_exoplanet_archive import NasaExoplanetArchive
p = NasaExoplanetArchive.query_criteria(table="pscomppars", select="*", 
                                        where="sy_dist < 70 and discoverymethod = 'Radial Velocity'")
p = p.to_pandas()
p = p.sort_values(by=['sy_dist'])
p = p.reset_index(drop = True)
p.to_csv('PS_2023.08.28.csv', index=False)
print(len(p))

763


## Get radius if missing

In [25]:
from myastrotools.tools import PlanetMass2Radius
p['M2R infered radius [Rearth]'] = np.nan
for i in range(len(p)):
    if np.isnan(p.loc[i]['pl_rade']):
        if not np.isnan(p.loc[i]['pl_bmasse']):
            p['M2R infered radius [Rearth]'][i] = PlanetMass2Radius(p.loc[i]['pl_bmasse'])
            

## SMA:
drop ones without sma 

In [27]:
ind = np.where(np.isnan(p['pl_orbsmax']))[0]
p = p.drop(ind)
p = p.reset_index(drop = True)
p

,pl_name,pl_letter,hostname,hd_name,hip_name,tic_id,disc_pubdate,disc_year,discoverymethod,disc_locale,...,sy_pmraerr1,sy_pmraerr2,sy_pmrastr,x,y,z,htm20,sky_coord.ra,sky_coord.dec,M2R infered radius [Rearth]
0,Proxima Cen b,b,Proxima Cen,,HIP 70890,TIC 388857263,2016-08,2016,Radial Velocity,Ground,...,0.085181,-0.085181,-3781.3100000&plusmn;0.0851812,-0.364683,-0.278755,-0.888426,-410784514,217.393466,-62.676182,NaN
1,eps Eri b,b,eps Eri,HD 22049,HIP 16537,TIC 118572803,2000-12,2000,Radial Velocity,Ground,...,0.567357,-0.567357,-975.376000&plusmn;0.567357,0.590488,0.790139,-0.164328,340435958,53.228431,-9.458172,NaN
2,GJ 887 c,c,GJ 887,HD 217987,HIP 114046,TIC 155315739,2020-06,2020,Radial Velocity,Ground,...,0.700000,NaN,6767.26&plusmn;0.70,0.788017,-0.189669,-0.585709,211708954,346.466827,-35.853069,NaN
3,GJ 887 b,b,GJ 887,HD 217987,HIP 114046,TIC 155315739,2020-06,2020,Radial Velocity,Ground,...,0.700000,NaN,6767.26&plusmn;0.70,0.788017,-0.189669,-0.585709,211708954,346.466827,-35.853069,NaN
4,Ross 128 b,b,Ross 128,,HIP 57548,TIC 325275315,2018-05,2017,Radial Velocity,Ground,...,0.136884,-0.136884,607.678000&plusmn;0.136884,-0.998475,0.053418,0.013950,-1767574165,176.937604,0.799290,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
740,HD 66141 b,b,HD 66141,HD 66141,HIP 39311,TIC 452867458,2012-12,2012,Radial Velocity,Ground,...,0.555695,-0.555695,-26.515300&plusmn;0.555695,-0.508113,0.860326,0.040743,-167361284,120.566282,2.335027,NaN
741,HD 143361 b,b,HD 143361,HD 143361,HIP 78521,TIC 255480497,2009-03,2008,Radial Velocity,Ground,...,0.103649,-0.103649,-156.502000&plusmn;0.103649,-0.352059,-0.621220,-0.700100,-1729539816,240.458841,-44.435057,NaN
742,HD 179079 b,b,HD 179079,HD 179079,HIP 94256,TIC 48917782,2009-09,2008,Radial Velocity,Ground,...,0.125075,-0.125075,-134.422000&plusmn;0.125075,0.305211,-0.951171,-0.046039,-824542996,287.790373,-2.638770,NaN
743,HD 72892 b,b,HD 72892,HD 72892,HIP 42098,TIC 405344271,2017-04,2016,Radial Velocity,Ground,...,0.061214,-0.061214,92.2361000&plusmn;0.0612145,-0.605701,0.755510,-0.249661,378693992,128.719555,-14.457431,NaN


## Get Spectral Type/Teff if missing:

In [46]:
np.where(np.isnan(p['st_teff']))

(array([345]),)

#### If there is no SpT, use Teff to estimate.  If there is no Teff, use SpT to estimate.  Using the Mamjek table
https://www.pas.rochester.edu/~emamajek/EEM_dwarf_UBVIJHK_colors_Teff.txt

In [48]:
def GetSpTNumber(j):
    sptletters = ['O','B','A','F','G','K','M','L','T','Y']
    sptnumbers = [0,1,2,3,4,5,6,7,8,9]
    ind = np.where([x['SpT'][j][0]==sptletters[i] for i in range(len(sptletters))])
    n = sptnumbers[ind[0][0]]
    n += float(x['SpT'][j][1])/10
    if '.' in x['SpT'][j]:
        n += 0.05
    return n

# Convert SpT to numbers to allow for interpolation:
x = pd.read_table('EEM_dwarf_UBVIJHK_colors_Teff.txt', delim_whitespace=True, comment='#',
                 nrows=118)
x['SpT Numbers'] = np.nan
for j in range(len(x)):
    x['SpT Numbers'][j] = GetSpTNumber(j)


In [49]:
# Create SpT/Teff lookup splines:
from scipy.interpolate import UnivariateSpline
GetTeffSpl = UnivariateSpline(x['SpT Numbers'], x['Teff'])
GetSpTSpl = UnivariateSpline(x['Teff'][::-1],x['SpT Numbers'][::-1])

In [51]:
# Now turn star SpT into numbers:
def GetSpTNumber(j):
    sptletters = ['O','B','A','F','G','K','M','L','T','Y']
    sptnumbers = [0,1,2,3,4,5,6,7,8,9]
    ind = np.where([p['st_spectype'][j][0]==sptletters[i] for i in range(len(sptletters))])
    try:
        n = sptnumbers[ind[0][0]]
    except IndexError:
        pass
    try:
        n += float(p['st_spectype'][j][1])/10
        if '.5' in p['st_spectype'][j]:
            n += 0.05
        return n
    except:
        pass


p['SpT Number'] = np.nan
for j in range(len(p)):
    if p['st_spectype'][j] == '':
        pass
    else:
        p['SpT Number'][j] = GetSpTNumber(j)

#### If there's no SpT infer it from Teff table:

In [83]:
p['Inferred SpT from Teff'] = np.nan
ind = np.where(np.isnan(p['SpT Number']))[0]

for i in ind:
    #print(i,p['SpT Number'][i],p['st_teff'][i])
    p['Inferred SpT from Teff'][i] = GetSpTSpl(p['st_teff'][i])  
    #print(p['Inferred SpT from Teff'][i])
    
ind = np.where(np.isnan(p['SpT Number']))[0]
for i in ind:
    p['SpT Number'][i] = p['Inferred SpT from Teff'][i]
print(np.where(np.isnan(p['SpT Number']))[0])

[]


#### If no Teff infer it from SpT table:

In [85]:
p['Inferred Teff from SpT'] = np.nan
p['StarTeff'] = p['st_teff']

ind = np.where(np.isnan(p['st_teff']))

for i in ind:
    p['Inferred Teff from SpT'][i] = GetTeffSpl(p['SpT Number'][i])  
    p['StarTeff'][i] = GetTeffSpl(p['SpT Number'][i]) 

### Compile radius:

In [90]:
p['PlanetRadiuse'] = p['pl_rade'].copy()
ind = np.where(np.isnan(p['pl_rade']))[0]
p['PlanetRadiuse'][ind] = p['M2R infered radius [Rearth]'][ind]
np.where(np.isnan(p['PlanetRadiuse']))

(array([], dtype=int64),)

## Star properties

#### If missing radius get from Mamjek table using Teff:

In [101]:
x = pd.read_table('EEM_dwarf_UBVIJHK_colors_Teff.txt', delim_whitespace=True, comment='#',
                 nrows=118)

for i in range(len(x)):
    if x['R_Rsun'][i] == '...':
        x['R_Rsun'][i] = np.nan
    x['R_Rsun'][i] = float(x['R_Rsun'][i])

from scipy.interpolate import UnivariateSpline
GetStarRadSpl = UnivariateSpline(x['Teff'][:110][::-1], x['R_Rsun'][:110][::-1])


In [109]:
p['StarRad'] = p['st_rad'].copy()

p['Inferred StarRad from Teff'] = np.nan
ind = np.where(np.isnan(p['st_rad']))[0]
for i in ind:
    p['Inferred StarRad from Teff'][i] = GetStarRadSpl(p['StarTeff'][i])
    p['StarRad'][i] = GetStarRadSpl(p['StarTeff'][i])

In [110]:
p.to_csv('GMagAO-X-target-list.csv',index=False)